In [1]:
%%time

import fitz  

pdf_path = "XCon Learning with Experts for Fine-grained Category Discovery.pdf"

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)  
    all_text = ""  

    for page in doc:
        all_text += page.get_text() 

    doc.close() 
    return all_text

pdf_text = extract_text_from_pdf(pdf_path)
print(pdf_text)


Y. FEI ET AL.: LEARNING WITH EXPERTS FOR FINE-GRAINED CATEGORY DISCOVERY 1
XCon: Learning with Experts for
Fine-grained Category Discovery
Yixin Fei1
yixin.feiyx@gmail.com
Zhongkai Zhao1
zhongkai.zhaok@gmail.com
Siwei Yang1,3
swyang.ac@gmail.com
Bingchen Zhao2,3
zhaobc.gm@gmail.com
1 Tongji University
Shanghai, China
2 University of Edinburgh,
Edinburgh, UK
3 LunarAI
Abstract
We address the problem of generalized category discovery (GCD) in this paper, i.e.
clustering the unlabeled images leveraging the information from a set of seen classes,
where the unlabeled images could contain both seen classes and unseen classes. The seen
classes can be seen as an implicit criterion of classes, which makes this setting different
from unsupervised clustering where the cluster criteria may be ambiguous. We mainly
concern the problem of discovering categories within a ﬁne-grained dataset since it is one
of the most direct applications of category discovery, i.e. helping experts discover novel
conce

In [2]:
with open("extracted_text.txt", "w", encoding="utf-8") as file:
    # Write the string to the file
    file.write(pdf_text)

In [3]:
%%time

from transformers import BartTokenizer, BartForConditionalGeneration
import math

def chunk_text(text, max_tokens=1024):
    chunks = []
    count = 0
    for i in range(0, len(text), max_tokens):
        chunks.append(text[i:i+max_tokens])
        count = count + 1
    return chunks, count

def summarize_chunks(chunks, max_sum_length):
    summaries = []
    tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
    model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
    for chunk in chunks:
        inputs = tokenizer(chunk, return_tensors="pt", max_length=1024, truncation=True)
        summary_ids = model.generate(inputs.input_ids, max_length=max_sum_length, early_stopping=True)
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        summaries.append(summary)
    return summaries

def concatenate_summaries(summaries):
    return " ".join(summaries)

max_text_length = 4096

long_text = pdf_text

# Step 1: Chunk the text
text_chunks, num_chunk = chunk_text(long_text)
max_sum_length = math.floor(max_text_length / num_chunk)

# Step 2: Summarize each chunk
chunk_summaries = summarize_chunks(text_chunks, max_sum_length)

# Step 3: Concatenate the summaries
final_summary = concatenate_summaries(chunk_summaries)

# Check summarized text is less than 4096 tokens
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
tokens = tokenizer.tokenize(final_summary)
number_of_tokens = len(tokens)
print("number_of_tokens: ", number_of_tokens)

print(final_summary)

number_of_tokens:  2634
Y. FEI ET AL.: LEARNING WITH EXPERTS FOR FINE-GRAINED CATEGORY DISCOVERY 1                XCon: Learning with Experts forFine-grained Category Discovery. We address the problem of generalized category discovery (GCD) in this paper. Expert-Contrastive Learning (XCon) is a novel method to mine useful information from the images. It uses k-means clustering and then performing contrastive learning on each sub-dataset to learn discriminative features. Experiments show a clear improved performance over the previous best methods, demonstrating the effectiveness of our method. ations are available, such as image recogni-tion [5] and object detection. However, collecting a dataset at scales like ImageNet or COCO is not always possible. The problem of generalized category discovery was recently formalized in [26] The aim is to discover categories within the unlabeled data by leveraging the information. Clusters formed by DINO features are mainly based on the class irrelev

In [6]:
%%time

from openai import OpenAI

client = OpenAI(api_key = 'YOUR-OPENAI-API-KEY')

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": final_summary},
    {"role": "user", "content": "Please convert the text into a presentation slides. Give the title and content for each slide, in the format of Slide 1: <title> <content>"}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content="Slide 1: Title\nLearning with Experts for Fine-grained Category Discovery\n\nSlide 2: Introduction\n- Addressing the problem of Generalized Category Discovery (GCD)\n- Introducing Expert-Contrastive Learning (XCon) for mining useful information from images\n- Utilizing k-means clustering and contrastive learning to learn discriminative features\n\nSlide 3: Motivation\n- Importance of Generalized Category Discovery in real-world applications\n- Contrasting unsupervised representation clustering with class-irrelevant cues vs. XCon's discriminative feature learning\n\nSlide 4: XCon Method\n- Expert-Contrastive Learning approach with k-means partitioning into expert sub-datasets\n- Eliminating negative influence of class-irrelevant cues for fine-grained category discovery\n- Using XCon for learning discriminative features and discovering new object categories\n\nSlide 5: Contrastive Learning in GCD\n- Leveraging contrastive learning for effective representati

In [7]:
message = completion.choices[0].message.content
print(message)

Slide 1: Title
Learning with Experts for Fine-grained Category Discovery

Slide 2: Introduction
- Addressing the problem of Generalized Category Discovery (GCD)
- Introducing Expert-Contrastive Learning (XCon) for mining useful information from images
- Utilizing k-means clustering and contrastive learning to learn discriminative features

Slide 3: Motivation
- Importance of Generalized Category Discovery in real-world applications
- Contrasting unsupervised representation clustering with class-irrelevant cues vs. XCon's discriminative feature learning

Slide 4: XCon Method
- Expert-Contrastive Learning approach with k-means partitioning into expert sub-datasets
- Eliminating negative influence of class-irrelevant cues for fine-grained category discovery
- Using XCon for learning discriminative features and discovering new object categories

Slide 5: Contrastive Learning in GCD
- Leveraging contrastive learning for effective representation learning in GCD
- Using k-means grouping on se

In [8]:
%%time

from pptx import Presentation

prs = Presentation()

# Split the response content into slides
slides_content = message.strip().split("\n\nSlide ")

# Process each slide
for slide_text in slides_content:
    # Split slide title from content
    title, *content = slide_text.split('\n')
    # Extract slide number and title
    _, title_text = title.split(": ", 1)
    
    # Create a new slide with a title and content layout
    slide_layout = prs.slide_layouts[1]  # Using layout 1 for title and content
    slide = prs.slides.add_slide(slide_layout)
    
    # Set the title and content for the slide
    title = slide.shapes.title
    content_box = slide.placeholders[1]
    
    title.text = title_text
    content_text = "\n".join(content).strip()  # Joining content lines
    content_box.text = content_text

# Save the presentation
prs.save('presentation.pptx')

CPU times: user 158 ms, sys: 16 ms, total: 174 ms
Wall time: 174 ms
